In [1]:
#Image Recognition using Pre-trained Model
#We'll use a pre-trained model like ResNet or EfficientNet for image recognition.

import torch
from torchvision import models, transforms
from PIL import Image

# Load a pre-trained model
model = models.resnet50(pretrained=True)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Image recognition function
def recognize_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
    _, predicted = outputs.max(1)
    return predicted.item()


C:\Users\adity\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\adity\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# NLP query function
def generate_response(query):
    inputs = tokenizer.encode(query, return_tensors='pt')
    outputs = model.generate(inputs, max_length=50, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [3]:
import requests

# URL for ImageNet class labels
url = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"

# Fetch and load the labels
response = requests.get(url)
labels = response.json()

# Extract class names
labels_list = [labels[str(i)][1] for i in range(1000)]

# Save to a text file
with open('imagenet_classes.txt', 'w') as f:
    for label in labels_list:
        f.write(label + '\n')

print("ImageNet class labels saved to imagenet_classes.txt")


ImageNet class labels saved to imagenet_classes.txt


In [4]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from io import BytesIO
import requests
import os

# Load the pre-trained model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Load ImageNet class labels
with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

def recognize_image(image_path):
    # Check if the image_path is a URL or a local file path
    if image_path.startswith('http://') or image_path.startswith('https://'):
        # Download the image from the URL
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        # Load the image from the local file system
        if not os.path.isfile(image_path):
            raise FileNotFoundError(f"Local file not found: {image_path}")
        image = Image.open(image_path)
    
    # Transform the image
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for ResNet
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Run the model
    with torch.no_grad():
        outputs = model(image)
    
    # Get the predicted class
    _, predicted = outputs.max(1)
    return predicted.item()

def generate_response(query_with_object):
    # Placeholder function for generating a response using NLP
    return f"Here is some information {query_with_object}."

def chatbot(image_path, user_query):
    # Recognize objects in the image
    object_id = recognize_image(image_path)
    
    # Map the object ID to a label from ImageNet
    recognized_object = labels[object_id]
    
    # Use NLP to generate a response
    query_with_object = f"{user_query} about a {recognized_object}"
    response = generate_response(query_with_object)
    
    return response


C:\Users\adity\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\adity\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# For a local file path
local_image_path = r"C:\Users\adity\Downloads\panzermkiv1200.jpg"  # Replace with the actual path to your local image
user_query = "Tell me more"
response = chatbot(local_image_path, user_query)
print(response)

# For an image URL
image_url = "https://g.foolcdn.com/image/?url=https%3A%2F%2Fg.foolcdn.com%2Feditorial%2Fimages%2F168693%2Ff-16.jpg&op=resize&w=700"  # Replace with the actual URL to your image
response = chatbot(image_url, user_query)
print(response)


Here is some information Tell me more about a tank.
Here is some information Tell me more about a warplane.


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/chatbot', methods=['POST'])
def chatbot_api():
    image = request.files['image']
    query = request.form['query']
    
    image_path = r"C:\Users\adity\Downloads\6aofsvaglm_Medium_WW226365.jpg"
    image.save(image_path)
    
    response = chatbot(image_path, query)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.3:5000
Press CTRL+C to quit


In [ ]:
curl -X POST -F 'image=C:\Users\adity\Downloads\6aofsvaglm_Medium_WW226365.jpg' -F 'query=Tell me more' http://localhost:5000/chatbot
